In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time
import random
import re
import pandas as pd
import feather
from sklearn import datasets
import sklearn

In [2]:
def separate_dataset(trainset, ratio = 0.5):
    datastring = []
    datatarget = []
    for i in range(len(trainset.data)):
        data_ = trainset.data[i].split('\n')
        data_ = list(filter(None, data_))
        data_ = random.sample(data_, int(len(data_) * ratio))
        for n in range(len(data_)):
            data_[n] = clearstring(data_[n])
        datastring += data_
        for n in range(len(data_)):
            datatarget.append(trainset.target[i])
    return datastring, datatarget

def clearstring(string):
    string = re.sub('[^A-Za-z0-9 ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = ' '.join(string)
    return string.lower()

trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)

train = pd.DataFrame(trainset.data,columns=['sequence'])
label = pd.DataFrame(trainset.target,columns=['labels'])

In [3]:
num_words = 50000
column_name = 'word_seg'
word_seq_len = 1800
embedding_vector = 200

In [4]:
from keras.preprocessing import text,sequence
from gensim.models import Word2Vec
num_words = 50000
import os
import numpy as np

def w2v_pad(df_train, col, maxlen_, victor_size):
    tokenizer = text.Tokenizer(num_words=num_words, filters='')
    tokenizer.fit_on_texts(list(df_train[col].values))
    
    train_ = sequence.pad_sequences(tokenizer.texts_to_sequences(df_train[col].values), maxlen=maxlen_)
    
    word_index = tokenizer.word_index
    
    count = 0
    nb_words = len(word_index)
    print(nb_words)
    all_data = df_train[col]
    file_name = './Word2Vec_' + col  +"_"+ '200' + '.model'
    if not os.path.exists(file_name):
        model = Word2Vec([[word for word in document.split(' ')] for document in all_data.values],
                        size=victor_size, window=5, iter=10, workers=11, seed=2018, min_count=2)
        model.save(file_name)
    else:
        model = Word2Vec.load(file_name)
    print('add word2vec')
    
    embedding_word2vec_matrix = np.zeros((nb_words + 1, victor_size))
    for word, i in word_index.items():
        embedding_vector = model[word] if word in model else None
        if embedding_vector is not None:
            count += 1
            embedding_word2vec_matrix[i] = embedding_vector
        else:
            unk_vec = np.random.random(victor_size) * 0.5
            unk_vec = unk_vec - unk_vec.mean()
            embedding_word2vec_matrix[i] = unk_vec    
    
    return train_, word_index, embedding_word2vec_matrix

Using TensorFlow backend.


In [5]:
train_, word2idx, word_embedding = w2v_pad(train, 'sequence', 40, 200)

20465
add word2vec


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [6]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

lb = LabelEncoder()
train_label = lb.fit_transform(label['labels'].values)
train_label = to_categorical(train_label)

In [13]:
import numpy as np

In [37]:
class Model():
    def __init__(self, size_layer, num_layers, embedding_matrix, 
                 dimension_output, learning_rate):
        def cells(size=128, reuse = False):
            return tf.nn.rnn_cell.BasicLSTMCell(num_units=size, reuse=reuse)
        
        
        
        
        

        
        
        
        def multihead_attention(queries, keys, values,
                                num_heads=8, 
                                dropout_rate=0,
                                training=True,
                                causality=False,
                                scope="multihead_attention"):
            '''Applies multihead attention. See 3.2.2
            queries: A 3d tensor with shape of [N, T_q, d_model].
            keys: A 3d tensor with shape of [N, T_k, d_model].
            values: A 3d tensor with shape of [N, T_k, d_model].
            num_heads: An int. Number of heads.
            dropout_rate: A floating point number.
            training: Boolean. Controller of mechanism for dropout.
            causality: Boolean. If true, units that reference the future are masked.
            scope: Optional scope for `variable_scope`.

            Returns
              A 3d tensor with shape of (N, T_q, C)  
            '''
            d_model = queries.get_shape().as_list()[-1]
            with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
                # Linear projections
                Q = tf.layers.dense(queries, d_model, use_bias=True) # (N, T_q, d_model)
                K = tf.layers.dense(keys, d_model, use_bias=True) # (N, T_k, d_model)
                V = tf.layers.dense(values, d_model, use_bias=True) # (N, T_k, d_model)

                # Split and concat
                Q_ = tf.concat(tf.split(Q, num_heads, axis=2), axis=0) # (h*N, T_q, d_model/h)
                K_ = tf.concat(tf.split(K, num_heads, axis=2), axis=0) # (h*N, T_k, d_model/h)
                V_ = tf.concat(tf.split(V, num_heads, axis=2), axis=0) # (h*N, T_k, d_model/h)

                # Attention
                outputs = scaled_dot_product_attention(Q_, K_, V_, causality, dropout_rate, training)

                # Restore shape
                outputs = tf.concat(tf.split(outputs, num_heads, axis=0), axis=2 ) # (N, T_q, d_model)

                # Residual connection
                outputs += queries

                # Normalize
                outputs = ln(outputs)

            return outputs


        def scaled_dot_product_attention(Q, K, V,
                                         causality=False, dropout_rate=0.,
                                         training=True,
                                         scope="scaled_dot_product_attention"):
            '''See 3.2.1.
            Q: Packed queries. 3d tensor. [N, T_q, d_k].
            K: Packed keys. 3d tensor. [N, T_k, d_k].
            V: Packed values. 3d tensor. [N, T_k, d_v].
            causality: If True, applies masking for future blinding
            dropout_rate: A floating point number of [0, 1].
            training: boolean for controlling droput
            scope: Optional scope for `variable_scope`.
            '''
            with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
                d_k = Q.get_shape().as_list()[-1]

                # dot product
                outputs = tf.matmul(Q, tf.transpose(K, [0, 2, 1]))  # (N, T_q, T_k)

                # scale
                outputs /= d_k ** 0.5


                # softmax
                outputs = tf.nn.softmax(outputs)
                attention = tf.transpose(outputs, [0, 2, 1])
                tf.summary.image("attention", tf.expand_dims(attention[:1], -1))

                # # query masking
                # outputs = mask(outputs, Q, K, type="query")

                # dropout
                outputs = tf.layers.dropout(outputs, rate=dropout_rate, training=training)

                # weighted sum (context vectors)
                outputs = tf.matmul(outputs, V)  # (N, T_q, d_v)

            return outputs






        def ln(inputs, epsilon = 1e-8, scope="ln"):
            '''Applies layer normalization. See https://arxiv.org/abs/1607.06450.
            inputs: A tensor with 2 or more dimensions, where the first dimension has `batch_size`.
            epsilon: A floating number. A very small number for preventing ZeroDivision Error.
            scope: Optional scope for `variable_scope`.

            Returns:
              A tensor with the same shape and data dtype as `inputs`.
            '''
            with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
                inputs_shape = inputs.get_shape()
                params_shape = inputs_shape[-1:]

                mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
                beta= tf.get_variable("beta", params_shape, initializer=tf.zeros_initializer(), dtype=tf.float64)
                gamma = tf.get_variable("gamma", params_shape, initializer=tf.ones_initializer(), dtype=tf.float64)
                normalized = (inputs - mean) / ( (variance + epsilon) ** (.5) )
                outputs = gamma * normalized + beta

            return outputs
        
        
        self.X = tf.placeholder(tf.int32, [None, 40])
        self.Y = tf.placeholder(tf.float32, [None, dimension_output])
        
        encoder_embedding = tf.Variable(embedding_matrix)
        encoder_embedded = tf.nn.embedding_lookup(encoder_embedding, self.X)
        
        
        out = multihead_attention(encoder_embedded,encoder_embedded,encoder_embedded)
        print('out',out)
        rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)])
        drop = tf.contrib.rnn.DropoutWrapper(rnn_cells, output_keep_prob=0.5)
        outputs, _ = tf.nn.dynamic_rnn(drop, out, dtype=tf.float64)
        print('outputs',outputs)
        W = tf.get_variable('w',shape=(128, dimension_output),dtype=tf.float64,initializer=tf.orthogonal_initializer())
        b = tf.get_variable('b',shape=(dimension_output),dtype=tf.float64,initializer=tf.zeros_initializer())
        self.logits = tf.matmul(outputs[:, -1], W) + b
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        correct_pred = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))        

In [38]:
size_layer = 128
num_layers = 1
embedded_size = 128
dimension_output = len(trainset.target_names)
learning_rate = 1e-3
maxlen = 50
batch_size = 128

In [39]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(size_layer,num_layers,word_embedding,dimension_output,learning_rate)
sess.run(tf.global_variables_initializer())

out Tensor("multihead_attention/ln/add_1:0", shape=(?, 40, 200), dtype=float64)
outputs Tensor("rnn/transpose_1:0", shape=(?, 40, 128), dtype=float64)


In [35]:
train_X, test_X, train_Y, test_Y = train_test_split(train_, train_label, test_size = 0.2)

In [36]:
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 5, 0, 0, 0
while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n'%(EPOCH))
        break
        
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    for i in range(0, (len(train_X) // batch_size) * batch_size, batch_size):
        batch_x = train_X[i:i+batch_size]
        batch_y = train_Y[i:i+batch_size]
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                           feed_dict = {model.X : batch_x, model.Y : batch_y})
        train_loss += loss
        train_acc += acc
    
    for i in range(0, (len(test_X) // batch_size) * batch_size, batch_size):
        batch_x = test_X[i:i+batch_size]
        batch_y = test_Y[i:i+batch_size]
        acc, loss = sess.run([model.accuracy, model.cost], 
                           feed_dict = {model.X : batch_x, model.Y : batch_y})
        test_loss += loss
        test_acc += acc
    
    train_loss /= (len(train_X) // batch_size)
    train_acc /= (len(train_X) // batch_size)
    test_loss /= (len(test_X) // batch_size)
    test_acc /= (len(test_X) // batch_size)
    
    if test_acc > CURRENT_ACC:
        print('epoch: %d, pass acc: %f, current acc: %f'%(EPOCH,CURRENT_ACC, test_acc))
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time()-lasttime)
    print('epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'%(EPOCH,train_loss,
                                                                                          train_acc,test_loss,
                                                                                          test_acc))
    EPOCH += 1

epoch: 0, pass acc: 0.000000, current acc: 0.611328
time taken: 5.605290412902832
epoch: 0, training loss: 0.692693, training acc: 0.552202, valid loss: 0.657425, valid acc: 0.611328

epoch: 1, pass acc: 0.611328, current acc: 0.625977
time taken: 4.566883563995361
epoch: 1, training loss: 0.550928, training acc: 0.709872, valid loss: 0.887401, valid acc: 0.625977

epoch: 2, pass acc: 0.625977, current acc: 0.716309
time taken: 4.715464115142822
epoch: 2, training loss: 0.253479, training acc: 0.891335, valid loss: 0.813973, valid acc: 0.716309

time taken: 4.680264234542847
epoch: 3, training loss: 0.072645, training acc: 0.972656, valid loss: 1.055491, valid acc: 0.709961

epoch: 4, pass acc: 0.716309, current acc: 0.722656
time taken: 4.6640589237213135
epoch: 4, training loss: 0.039769, training acc: 0.986742, valid loss: 1.155890, valid acc: 0.722656

epoch: 5, pass acc: 0.722656, current acc: 0.724609
time taken: 4.511036157608032
epoch: 5, training loss: 0.016603, training acc: 